## Load Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## download in below root directory data from kaggle at https://www.kaggle.com/datasets/vaidiknakrani/onfieldalignment 

root_path = 'root_dir_path' # update path to your directory where you downloaded data.
label = pd.read_csv('path_label_csv') # update this path according to your environment.

X, y = [], label['angle(radian)'].values.reshape(-1,1)
for p in label['img_path'].values:
    X.append(cv2.imread(root_path + '/images/' + p))
X = np.asarray(X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)

In [ ]:
data = tf.data.Dataset.from_tensor_slices((X, y))

data = data.shuffle(200).batch(8)

train_n = int(len(data)*0.80)
print(f'train data have {train_n} batches and val data have {len(data)-train_n} batches...')

train_data = data.take(train_n)
val_data = data.skip(train_n)

train_data = train_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train data have 32 batches and val data have 9 batches...


## define model

In [ ]:
input = tf.keras.layers.Input((500,500,3))
input = tf.keras.layers.Rescaling(1./255)(input)


bmodel = tf.keras.applications.MobileNetV3Small(input_shape=(500,500,3), include_top=False)
for l in bmodel.layers[-5:]:
  l.trainable = True

o = bmodel(input, training=False)
o = tf.keras.layers.GlobalAveragePooling2D()(o)
o = tf.keras.layers.Dropout(0.05)(o)
o = tf.keras.layers.Dense(8, activation='relu')(o)
o = tf.keras.layers.Dense(4, activation='relu')(o)
o = tf.keras.layers.Dense(2, activation='relu')(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)
m = tf.keras.Model(inputs=input, outputs=o)
m.summary()

def lr_sch(e, lr):
  if e<=50:
    return lr
  else:
    return lr*tf.math.exp(-0.0002)

m.compile(optimizer='sgd', loss='mae')
callback = tf.keras.callbacks.LearningRateScheduler(lr_sch)
h = m.fit(train_data, epochs=250, callbacks=[callback], validation_data=val_data)

import matplotlib.pyplot as plt
plt.plot(range(250), h.history['loss'])
plt.plot(range(250), h.history['val_loss'])

In [ ]:
m.save(root_path + '/modelv25.h5')

In [ ]:
from google.colab.patches import cv2_imshow
def put_text(img, label, arr):
  height, width, _ = img.shape
  label = 'Degree : '+label
  # Define the text to display

  font = cv2.FONT_HERSHEY_TRIPLEX
  font_scale = 0.8
  thickness = 1

  # Get the size of the label text
  text_size, _ = cv2.getTextSize(label, font, font_scale, thickness)

  # Calculate the coordinates for the label and rectangle
  label_x = int((img.shape[1] - text_size[0]) / 2)
  label_y = img.shape[0] - 10
  rect_x = int((img.shape[1] - text_size[0]) / 2)-20
  rect_y = label_y + 10
  rect_w = text_size[0]
  rect_h = text_size[1] + 10

  # Draw the rectangle and label
  cv2.rectangle(img, (width//2-90, 0), (width//2+100, rect_h+10), (255, 255, 255), -1)
  cv2.putText(img, label, (width//2-85, 23), font, font_scale, (255, 0, 0), thickness)

  if p==1:
    cv2.arrowedLine(img, (width//2, rect_h+15), (width//2 - p*200//20, rect_h+15), (0,0,255), 2)
  else:
    cv2.arrowedLine(img, (width//2, rect_h+15), (width//2 + p*200//20, rect_h+15), (0,0,255), 2)

  return img

## Test on video

In [ ]:
v = cv2.VideoCapture(r"path_of_video")
result = cv2.VideoWriter('path_of_output_video(avi)',
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, (600, 400))
n=1
while v.isOpened():
    cap, f = v.read()
    if cap and n<90:
        print(1)
        f = cv2.resize(f, (500,500))
        f = np.expand_dims(f, axis=0)
        p = m.predict(f, verbose=0)

        p = scaler.inverse_transform(np.asarray(p).reshape(-1,1))[0]
        p = int(p*180/3.14)
        f = np.squeeze(f, 0)

        if p<0:
          f = put_text(f, str(p), -1)
        else:
          f = put_text(f, str(p), 1)
          
        f = cv2.resize(f, (600, 400))
        if n>20:
          result.write(f)

        n+=1
    else:
        break
    
      
v.release()
result.release()